In [1]:
import autogen
from autogen.agentchat.contrib.agent_builder import AgentBuilder
import os
from dotenv import load_dotenv
import json

load_dotenv()

True

In [19]:
env_var = [
    {
        'model': 'gpt-4',
        'api_key': os.getenv('OPENAI_API_KEY')
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': os.getenv('OPENAI_API_KEY')
    }
]

In [28]:
config_name = 'autogen_config.json'

config_list = autogen.config_list_from_json(
        env_or_file=config_name,
        filter_dict={
            "model": {
                "gpt-4",
                "gpt-3.5-turbo",
            }
        }
    )

assert len(config_list) > 0 
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

default_llm_config = {'temperature': 0}

models to use:  ['gpt-4', 'gpt-3.5-turbo']


In [29]:
builder = AgentBuilder(config_path=config_name)


In [31]:
building_task = "Find a paper on arxiv by programming, and analyze its application in some domain..."
agent_list, agent_configs = builder.build(building_task, default_llm_config)

Generating agents...
Data_scientist,Research_analyst,Computer_programmer are generated.
Preparing configuration for Data_scientist...
Preparing configuration for Research_analyst...
Preparing configuration for Computer_programmer...
Creating agent Data_scientist with backbone gpt-4...
Creating agent Research_analyst with backbone gpt-4...
Creating agent Computer_programmer with backbone gpt-4...
Adding user console proxy...


In [ ]:
group_chat = autogen.GroupChat(agents=agent_list, messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=group_chat, llm_config={"config_list": config_list, **default_llm_config})
agent_list[0].initiate_chat(
    manager,
    message='Find a recent paper about gpt-4 on arxiv...'
)


In [2]:
from openai import OpenAI

client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))

In [3]:
model = 'gpt-4'

In [4]:
openai_client.chat.completions.create(model=model, messages=messages)

NameError: name 'openai_client' is not defined

In [38]:
from enum import Enum


class EmailMessage:
    body: str
    subject: str
    from_: str
    to: str
    id: int
    
class DialogState(Enum):
    NONE = 1
    UNHANDLED_REQUEST = 2
    

In [39]:
state = DialogState.NONE

def email_notifier(email: EmailMessage):
    if email.id == 1:
        state = DialogState

SyntaxError: incomplete input (1866276236.py, line 5)

In [51]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType

# Initialize the large language model
# llm = OpenAI(temperature=0, openai_api_key=os.getenv('OPENAI_API_KEY'), model_name="gpt-4-1106-preview")
# llm = ChatOpenAI(temperature=0, openai_api_key=os.getenv('OPENAI_API_KEY'), model_name="gpt-4-1106-preview")

# # Initialize the conversation chain with ConversationBufferMemory
# conversation_buf = ConversationChain(llm=llm, memory=ConversationBufferMemory())


# response = conversation_buf.predict(input="Hi there!")


# # Start the conversation
# # response = conversation_buf("Good morning AI!")
print(response)


Hello! It's great to have a chance to chat with you. What's on your mind today? If you have any questions or topics you'd like to explore, feel free to ask, and I'll do my best to provide you with detailed information. If there's something I don't know, I'll be sure to let you know.


In [54]:
sales_manager_system_message = """
You are Sales Manager(SM) at manufacturer details dealer.  Also there are Research Analyst (RA), Customer Service Reprisentative (CSR).  The business model of the company is very simple: Customers ask to buy parts, and a CSR (Customer Service Representative) looks at the request to see what parts the customer needs. Then, an RA (Research Analyst) finds the best prices for these parts online and tells the CSR. The CSR makes an offer and sends it to the customer. The customer can either buy the parts or have questions, and the CSR helps with these questions.

Description of the area of responsibility for each employee under your supervision.

Agent 1. Customer Service Reprisentative (CSR)
- classify customer request: extract details info from client email text body: brand name, part number, amount, model, serial number etc.
- answer on technical questions
- ping customer when it not responds
- generate commercial offer

Agent 2. Research Analyst (RA)
- search at CRM database: search detail by brand name or model or part number at company CRM database
- search at Google: search details at google with query

Your responsibility as a Sales Manager is to make decisions based on the information provided and to assign tasks to your subordinates. The task should be set in the following format:

Agent: responsible employee
State: this is one or more words in CamelCase that describe the current state
Task: a detailed description of the task in 1-2 sentences, which will be understandable to the employee.

[Example 1]
Agent: Customer Service Reprisentative (CSR)
State: PartClassification
Task: Customer sends email, you need to classify it and select information or generate response message.
"""

sales_manager_agent = initialize_agent(
    tools=[],
    llm=ChatOpenAI(model="gpt-4-1106-preview", temperature=0),
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=1,
    early_stopping_method="generate",
    agent_kwargs={
        'system_message': sales_manager_system_message
    }
)


In [60]:
response = sales_manager_agent.run({
    "input": """
As a Sales Manager, please assign a task to one of the agents.

A customer inquiry has been received via email.

[Customer Request]
Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ 
Date: Thu, 30 Nov 2023 15:50:44 +0400 
From: Ramees Khan | AREC <ramees@arecuae.com>
To: info@famaga.ae  Dear Sir/Ma’am

[History of interaction]
Email id: 1
Is first touch with this customer or company: yes
""",
    "chat_history": []
    
})



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Agent: Customer Service Representative (CSR)\nState: InitialInquiryHandling\nTask: A new customer inquiry has been received. Please classify the request by extracting the brand name, part number, quantity, and any other relevant details from the email. Then, prepare and send an initial response acknowledging the receipt of the inquiry and informing the customer that we will provide the requested information shortly."
}
```

> Finished chain.


In [62]:
print(response)

Agent: Customer Service Representative (CSR)
State: InitialInquiryHandling
Task: A new customer inquiry has been received. Please classify the request by extracting the brand name, part number, quantity, and any other relevant details from the email. Then, prepare and send an initial response acknowledging the receipt of the inquiry and informing the customer that we will provide the requested information shortly.


In [64]:
response  = sales_manager_agent.run({
    
    "input": "What email was of previous message?",
    "chat_history": []
})

response



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "I'm sorry, but I cannot retrieve previous messages or emails. How can I assist you with your current inquiry?"
}
```

> Finished chain.


"I'm sorry, but I cannot retrieve previous messages or emails. How can I assist you with your current inquiry?"

In [5]:
from langchain.agents import create_vectorstore_agent
from langchain.vectorstores import Chroma
from langchain.agents.agent_toolkits import VectorStoreToolkit, VectorStoreInfo

In [72]:
Chroma?

Init signature:
Chroma(
    collection_name: 'str' = 'langchain',
    embedding_function: 'Optional[Embeddings]' = None,
    persist_directory: 'Optional[str]' = None,
    client_settings: 'Optional[chromadb.config.Settings]' = None,
    collection_metadata: 'Optional[Dict]' = None,
    client: 'Optional[chromadb.Client]' = None,
    relevance_score_fn: 'Optional[Callable[[float], float]]' = None,
) -> 'None'
Docstring:     
`ChromaDB` vector store.

To use, you should have the ``chromadb`` python package installed.

Example:
    .. code-block:: python

            from langchain.vectorstores import Chroma
            from langchain.embeddings.openai import OpenAIEmbeddings

            embeddings = OpenAIEmbeddings()
            vectorstore = Chroma("langchain_store", embeddings)
Init docstring: Initialize with a Chroma client.
File:           ~/projs/components_agent_sales/venv/lib/python3.11/site-packages/langchain/vectorstores/chroma.py
Type:           ABCMeta
Subclasses:     

In [6]:
chroma = Chroma(collection_name='sales_agent', persist_directory='./data/chroma', embedding_function='text-embedding-ada-002')

In [89]:

sales_manager_agent = create_vectorstore_agent(
    llm=ChatOpenAI(model="gpt-4-1106-preview", temperature=0),
    toolkit=VectorStoreToolkit(
        vectorstore_info=VectorStoreInfo(
            vectorstore=chroma,
            name='SA store',
            description='Store chat'
        )
    ),
    verbose=True,
    prefix=sales_manager_system_message
)

In [94]:
import langchain

langchain.debug = True

In [95]:
response = sales_manager_agent.run({
    "input": """
As a Sales Manager, please assign a task to one of the agents.

A customer inquiry has been received via email.

[Customer Request]
Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ 
Date: Thu, 30 Nov 2023 15:50:44 +0400 
From: Ramees Khan | AREC <ramees@arecuae.com>
To: info@famaga.ae  Dear Sir/Ma’am

[History of interaction]
Email id: 1
Is first touch with this customer or company: yes
"""    
})
response

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "\nAs a Sales Manager, please assign a task to one of the agents.\n\nA customer inquiry has been received via email.\n\n[Customer Request]\nSubject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ \nDate: Thu, 30 Nov 2023 15:50:44 +0400 \nFrom: Ramees Khan | AREC <ramees@arecuae.com>\nTo: info@famaga.ae  Dear Sir/Ma’am\n\n[History of interaction]\nEmail id: 1\nIs first touch with this customer or company: yes\n"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "\nAs a Sales Manager, please assign a task to one of the agents.\n\nA customer inquiry has been received via email.\n\n[Customer Request]\nSubject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ \nDate: Thu, 30 Nov 2023 15:50:44 +0400 \nFrom: Ramees Khan | AREC <ramees@arecuae.com>\nTo: info@famaga.ae  Dear Sir/Ma’am\n\n[History of interaction]\nEmail id: 1\n

'Tasks have been assigned to the CSR to classify the customer request and to the RA to research the best prices for the Firetrol Battery Charger AS-2001.'

In [92]:
response

'Tasks have been assigned to the CSR to classify the customer request and to the RA to research the best prices for the Firetrol Battery Charger AS-2001.'

In [93]:
sales_manager_agent.run('Who was the message sender?')



> Entering new AgentExecutor chain...
The question seems to be asking for the identity of a message sender, but there is no context or specific message provided to identify the sender. As a Sales Manager in this scenario, I would need more information to determine who sent a message.

Thought: I need to clarify the context or get more details about the message to identify the sender.

Final Answer: To identify the message sender, I would need more information about the specific message in question. Please provide additional context or details about the message.

> Finished chain.


'To identify the message sender, I would need more information about the specific message in question. Please provide additional context or details about the message.'

In [ ]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chain = RetrievalQA.from_chain_type(
            self.llm, retriever=self.vectorstore.as_retriever()
        )


In [ ]:
#vectorstore retriever

def get_relevant_documents(self, query: str) -> List[Document]:
        if self.search_type == "similarity":
            docs = self.vectorstore.similarity_search(query, **self.search_kwargs)
        elif self.search_type == "similarity_score_threshold":
            docs_and_similarities = (
                self.vectorstore.similarity_search_with_relevance_scores(
                    query, **self.search_kwargs
                )
            )
            docs = [doc for doc, _ in docs_and_similarities]
        elif self.search_type == "mmr":
            docs = self.vectorstore.max_marginal_relevance_search(
                query, **self.search_kwargs
            )
        else:
            raise ValueError(f"search_type of {self.search_type} not allowed.")
        return docs


In [97]:
human_prompt = """
Human: \nYou are Sales Manager(SM) at manufacturer details dealer.  Also there are Research Analyst (RA), Customer Service Reprisentative (CSR).  The business model of the company is very simple: Customers ask to buy parts, and a CSR (Customer Service Representative) looks at the request to see what parts the customer needs. Then, an RA (Research Analyst) finds the best prices for these parts online and tells the CSR. The CSR makes an offer and sends it to the customer. The customer can either buy the parts or have questions, and the CSR helps with these questions.\n\nDescription of the area of responsibility for each employee under your supervision.\n\nAgent 1. Customer Service Reprisentative (CSR)\n- classify customer request: extract details info from client email text body: brand name, part number, amount, model, serial number etc.\n- answer on technical questions\n- ping customer when it not responds\n- generate commercial offer\n\nAgent 2. Research Analyst (RA)\n- search at CRM database: search detail by brand name or model or part number at company CRM database\n- search at Google: search details at google with query\n\nYour responsibility as a Sales Manager is to make decisions based on the information provided and to assign tasks to your subordinates. The task should be set in the following format:\n\nAgent: responsible employee\nState: this is one or more words in CamelCase that describe the current state\nTask: a detailed description of the task in 1-2 sentences, which will be understandable to the employee.\n\n[Example 1]\nAgent: Customer Service Reprisentative (CSR)\nState: PartClassification\nTask: Customer sends email, you need to classify it and select information or generate response message.\n\n\nSA store: Useful for when you need to answer questions about SA store. Whenever you need information about Store chat you should ALWAYS use this. Input should be a fully formed question.\nSA store_with_sources: Useful for when you need to answer questions about SA store and the sources used to construct the answer. Whenever you need information about Store chat you should ALWAYS use this.  Input should be a fully formed question. Output is a json serialized dictionary with keys `answer` and `sources`. Only use this tool if the user explicitly asks for sources.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [SA store, SA store_with_sources]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: \nAs a Sales Manager, please assign a task to one of the agents.\n\nA customer inquiry has been received via email.\n\n[Customer Request]\nSubject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ \nDate: Thu, 30 Nov 2023 15:50:44 +0400 \nFrom: Ramees Khan | AREC <ramees@arecuae.com>\nTo: info@famaga.ae  Dear Sir/Ma’am\n\n[History of interaction]\nEmail id: 1\nIs first touch with this customer or company: yes\n\nThought:
"""

print(human_prompt)


Human: 
You are Sales Manager(SM) at manufacturer details dealer.  Also there are Research Analyst (RA), Customer Service Reprisentative (CSR).  The business model of the company is very simple: Customers ask to buy parts, and a CSR (Customer Service Representative) looks at the request to see what parts the customer needs. Then, an RA (Research Analyst) finds the best prices for these parts online and tells the CSR. The CSR makes an offer and sends it to the customer. The customer can either buy the parts or have questions, and the CSR helps with these questions.

Description of the area of responsibility for each employee under your supervision.

Agent 1. Customer Service Reprisentative (CSR)
- classify customer request: extract details info from client email text body: brand name, part number, amount, model, serial number etc.
- answer on technical questions
- ping customer when it not responds
- generate commercial offer

Agent 2. Research Analyst (RA)
- search at CRM database: se

In [38]:
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter

In [99]:
ConversationalRetrievalChain?

Init signature:
ConversationalRetrievalChain(
    *,
    memory: Optional[langchain_core.memory.BaseMemory] = None,
    callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    verbose: bool = None,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    combine_docs_chain: langchain.chains.combine_documents.base.BaseCombineDocumentsChain,
    question_generator: langchain.chains.llm.LLMChain,
    output_key: str = 'answer',
    rephrase_question: bool = True,
    return_source_documents: bool = False,
    return_generated_question: bool = False,
    get_chat_history: Optional[Callable[[List[Union[Tuple[str, str], langchain_core.messages.base.BaseMessage]]], str]] = None,
    response_if_no_docs_found: Optional[str] = None,
    retriever: langchain_core.retrievers.BaseRetrieve

In [14]:
email = """
Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ Date: Thu, 30 Nov 2023 15:50:44 +0400
From: Ramees Khan | AREC <ramees@arecuae.com>
To: info@famaga.ae Dear Sir/Ma’am,
We’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.
Please quote your best discounted prices and availability.
I have attached a picture below for your reference.  Thanking you.
Best Regards,
RAMEES KHAN
General Manager
M: +971-52-6927466 | E: ramees@arecuae.com
AL RAMIZ ELECT. CONT. LLC
A: Industrial Area 17, Sharjah - United Arab Emirates
A: P.O Box: 96286, Sharjah
T: +971-6-5356891
E: info@arecuae.com | W: www.arecuae.com
 Transformers/Maintenance, Erection & Testing | Rewinding – Transformers, Motors & Generators | Cables – Laying, Termination, Testing – HT & LT | Distribution Boards – MDB, SMDB, DB | Generator Alternator – Repair & Maintenance, Bus Bar Modification & Testing | MV, LV Electrical Panel – Installation & Testing | Hiring of all types of Testing Instruments – Oil Filter Plants, Oil Tanks, Vacuum Pumps, Crimping Tools & Generators.
Disclaimer: This e-mail and any attachments may contain confidential and privileged information. If you are not the intended recipient, please notify the sender immediately by return e-mail, delete this e-mail and destroy any copies. Any dissemination or use of this information by a person other than the intended recipient is unauthorized and may be illegal.
"""

In [15]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
# text = "This is a test query."
# query_result = embeddings.embed_query(text)


In [17]:
chroma = Chroma(collection_name='sales_agent', persist_directory='./data/chroma', embedding_function=embeddings)

In [39]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./emails.txt", encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
for i, text in enumerate(texts):
    text.metadata["source"] = f"{i}-pl"
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

In [18]:
fixed_resp = "I don't know"
answer = "I know the answer!"

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

memory = ConversationBufferMemory(
        k=1, output_key="answer", memory_key="chat_history", return_messages=True
)
   

In [32]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\
Make sure to avoid using any unclear pronouns.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)
condense_question_chain = LLMChain(
    llm=llm,
    prompt=CONDENSE_QUESTION_PROMPT,
)

In [40]:
qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        memory=memory,
        retriever=docsearch.as_retriever(),
        # condense_question_prompt=CONDENSE_QUESTION_PROMPT,
        # question_generator=condense_question_chain,
        # return_source_documents=True,
        # rephrase_question=False,
        # response_if_no_docs_found=fixed_resp,
        verbose=True,
    )
    # got = await qa_chain.acall("What is the answer?")
    # assert got["chat_history"][1].content == fixed_resp
    # assert got["answer"] == fixed_resp


In [21]:
got = await qa_chain.acall(f"Rumember this email: {email} and answer REMEMBERED.")
got['answer']



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------

Human: Rumember this email: 
Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ Date: Thu, 30 Nov 2023 15:50:44 +0400
From: Ramees Khan | AREC <ramees@arecuae.com>
To: info@famaga.ae Dear Sir/Ma’am,
We’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.
Please quote your best discounted prices and availability.
I have attached a picture below for your reference.  Thanking you.
Best Regards,
RAMEES KHAN
General Manager
M: +971-52-6927466 | E: ramees@arecuae.com
AL RAMIZ ELECT. CONT. LLC
A: Industrial Area 17, Sharjah - United Arab Emirates
A: P.O Box: 96286, Sharjah
T: +971-6-5356891
E: info@arecuae.com | W: www.arecuae.com
 Tra

'REMEMBERED.'

In [26]:
import langchain
langchain.debug = True

In [41]:
got = await qa_chain.acall({ 'question': 'Get subject from email.'})
got['answer']

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": "Get subject from email.",
  "chat_history": "\nHuman: Rumember this email: \nSubject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ Date: Thu, 30 Nov 2023 15:50:44 +0400\nFrom: Ramees Khan | AREC <ramees@arecuae.com>\nTo: info@famaga.ae Dear Sir/Ma’am,\nWe’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.\nPlease quote your best discounted prices and availability.\nI have attached a picture below for your reference.  Thanking you.\nBest Regards,\nRAMEES KHAN\nGeneral Manager\nM: +971-52-6927466 | E: ramees@arecuae.com\nAL RAMIZ ELECT. CONT. LLC\nA: Industrial Area 17, Sharjah - United Arab Emirates\nA: P.O Box: 96286, Sharjah\nT: +971-6-5356891\nE: info@arecuae.com | W: www.arecuae.com\n Transformers/Mainte

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


[chain/start] [1:chain:ConversationalRetrievalChain > 5:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 5:chain:StuffDocumentsChain > 6:chain:LLMChain] Entering Chain run with input:
{
  "question": "What is the subject line of the email from Ramees Khan of AREC?",
  "context": "Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ Date: Thu, 30 Nov 2023 15:50:44 +0400\nFrom: Ramees Khan | AREC <ramees@arecuae.com>\nTo: info@famaga.ae Dear Sir/Ma’am,\nWe’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.\nPlease quote your best discounted prices and availability.\nI have attached a picture below for your reference.  Thanking you.\nBest Regards,\nRAMEES KHAN\nGeneral Manager\nM: +971-52-6927466 | E: ramees@arecuae.com\nAL RAMIZ ELECT. CONT. LLC\nA: Industrial Area 17, Sharjah - United Arab Emirates\nA: P.O Box: 96286, Sharjah\nT: +971-6-5356891\

'The subject line of the email from Ramees Khan of AREC is "Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ".'

In [ ]:
from langchain.chains import (
                StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
            )
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI

            combine_docs_chain = StuffDocumentsChain(...)
            vectorstore = ...
            retriever = vectorstore.as_retriever()

            # This controls how the standalone question is generated.
            # Should take `chat_history` and `question` as input variables.
            template = (
                "Combine the chat history and follow up question into "
                "a standalone question. Chat History: {chat_history}"
                "Follow up question: {question}"
            )
            prompt = PromptTemplate.from_template(template)
            llm = OpenAI()
            question_generator_chain = LLMChain(llm=llm, prompt=prompt)
            chain = ConversationalRetrievalChain(
                combine_docs_chain=combine_docs_chain,
                retriever=retriever,
                question_generator=question_generator_chain,
            )


In [ ]:
def _get_chat_history(chat_history: List[CHAT_TURN_TYPE]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "Human: " + dialogue_turn[0]
            ai = "Assistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer


#

* чтобы получить историю чата, нужно получить 

In [46]:
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [ ]:
memory = ConversationBufferMemory(
    k=1, output_key="answer", memory_key="chat_history", return_messages=True
)

In [67]:
inputs = {
 'question': 'What about email message?'   
}

In [45]:
from langchain.chains import StuffDocumentsChain


prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

llm_chain = LLMChain(
        llm=llm, prompt=PROMPT, verbose=True
)
stuff_documents_chain = StuffDocumentsChain(
        llm_chain=llm_chain,
        document_variable_name='context',
        verbose=True
    )


### Condense

In [47]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

condense_question_chain = LLMChain(
    llm=llm,
    prompt=CONDENSE_QUESTION_PROMPT,
    verbose=True        
)


In [71]:
from langchain_core.messages import BaseMessage


_ROLE_MAP = {"human": "Human: ", "ai": "Assistant: "}


def _get_chat_history(chat_history: list) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "Human: " + dialogue_turn[0]
            ai = "Assistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer



In [49]:
memory.memory_variables

['chat_history']

In [58]:
input_keys = ["question", "chat_history"]


In [62]:
set(['question']).difference(['chat_history'])

{'question'}

In [69]:
if not isinstance(inputs, dict):
    _input_keys = set(input_keys)
    if memory is not None:
        _input_keys = _input_keys.difference(memory.memory_variables)
        if len(_input_keys) != 1:
            raise ValueError(
                    f"A single string input was passed in, but this chain expects "
                    f"multiple inputs ({_input_keys}). When a chain expects "
                    f"multiple inputs, please call it by passing in a dictionary, "
                    "eg `chain({'foo': 1, 'bar': 2})`"
                )
        inputs = {list(_input_keys)[0]: inputs}
if memory is not None:
    external_context = memory.load_memory_variables(inputs)
    inputs = dict(inputs, **external_context)

inputs

{'question': 'What about email message?',
 'chat_history': [HumanMessage(content='Rumember this email: \nSubject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ\u2028Date: Thu, 30 Nov 2023 15:50:44 +0400\nFrom: Ramees Khan | AREC <ramees@arecuae.com>\nTo: info@famaga.ae Dear Sir/Ma’am,\nWe’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.\nPlease quote your best discounted prices and availability.\nI have attached a picture below for your reference.\u2028\u2028Thanking you.\nBest Regards,\nRAMEES KHAN\nGeneral Manager\nM: +971-52-6927466 | E: ramees@arecuae.com\nAL RAMIZ ELECT. CONT. LLC\nA: Industrial Area 17, Sharjah - United Arab Emirates\nA: P.O Box: 96286, Sharjah\nT: +971-6-5356891\nE: info@arecuae.com | W: www.arecuae.com\n\u2028Transformers/Maintenance, Erection & Testing | Rewinding – Transformers, Motors & Generators | Cables – Laying, Termination, Testing – HT & LT | Distribution Boards – MDB, SMDB, DB | G

In [ ]:
print(_get_chat_history(inputs['chat_history']))

In [68]:
memory.load_memory_variables(inputs)

{'chat_history': [HumanMessage(content='Rumember this email: \nSubject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ\u2028Date: Thu, 30 Nov 2023 15:50:44 +0400\nFrom: Ramees Khan | AREC <ramees@arecuae.com>\nTo: info@famaga.ae Dear Sir/Ma’am,\nWe’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.\nPlease quote your best discounted prices and availability.\nI have attached a picture below for your reference.\u2028\u2028Thanking you.\nBest Regards,\nRAMEES KHAN\nGeneral Manager\nM: +971-52-6927466 | E: ramees@arecuae.com\nAL RAMIZ ELECT. CONT. LLC\nA: Industrial Area 17, Sharjah - United Arab Emirates\nA: P.O Box: 96286, Sharjah\nT: +971-6-5356891\nE: info@arecuae.com | W: www.arecuae.com\n\u2028Transformers/Maintenance, Erection & Testing | Rewinding – Transformers, Motors & Generators | Cables – Laying, Termination, Testing – HT & LT | Distribution Boards – MDB, SMDB, DB | Generator Alternator – Repair & Maintenance

In [ ]:
memory.save_context(outputs)

In [75]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./emails.txt", encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
for i, text in enumerate(texts):
    text.metadata["source"] = f"{i}-pl"
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

In [76]:
retriever = docsearch.as_retriever()

In [89]:
import langchain

langchain.debug = True

In [77]:
question = inputs["question"]
chat_history_str = _get_chat_history(inputs["chat_history"])

In [80]:
if chat_history_str:
    new_question = condense_question_chain.run(question=question, chat_history=chat_history_str)
else:
    new_question = question
    
new_question



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Rumember this email: 
Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ Date: Thu, 30 Nov 2023 15:50:44 +0400
From: Ramees Khan | AREC <ramees@arecuae.com>
To: info@famaga.ae Dear Sir/Ma’am,
We’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.
Please quote your best discounted prices and availability.
I have attached a picture below for your reference.  Thanking you.
Best Regards,
RAMEES KHAN
General Manager
M: +971-52-6927466 | E: ramees@arecuae.com
AL RAMIZ ELECT. CONT. LLC
A: Industrial Area 17, Sharjah - United Arab Emirates
A: P.O Box: 96286, Sharjah
T: +971-6-5356891
E: info@arecuae.com | W: www.arecuae.com
 Transformers/Maintenance, Erection & Testing | Rewinding – Transformers, 

'Can you provide the content of the email message from Ramees Khan of AREC?'

In [81]:
docs = retriever.get_relevant_documents(question)
docs

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(page_content='Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ\u2028Date: Thu, 30 Nov 2023 15:50:44 +0400\nFrom: Ramees Khan | AREC <ramees@arecuae.com>\nTo: info@famaga.ae Dear Sir/Ma’am,\nWe’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.\nPlease quote your best discounted prices and availability.\nI have attached a picture below for your reference.\u2028\u2028Thanking you.\nBest Regards,\nRAMEES KHAN\nGeneral Manager\nM: +971-52-6927466 | E: ramees@arecuae.com\nAL RAMIZ ELECT. CONT. LLC\nA: Industrial Area 17, Sharjah - United Arab Emirates\nA: P.O Box: 96286, Sharjah\nT: +971-6-5356891\nE: info@arecuae.com | W: www.arecuae.com\n\u2028Transformers/Maintenance, Erection & Testing | Rewinding – Transformers, Motors & Generators | Cables – Laying, Termination, Testing – HT & LT | Distribution Boards – MDB, SMDB, DB | Generator Alternator – Repair & Maintenance, Bus Bar Modification & Testing | MV, 

In [91]:
from typing import Dict, Any
# return self._reduce_tokens_below_limit(docs)

output_key = 'answer'
output: Dict[str, Any] = {}

new_inputs = inputs.copy()
new_inputs["question"] = new_question
new_inputs["chat_history"] = chat_history_str
answer = stuff_documents_chain.run(input_documents=docs, **new_inputs)
output[output_key] = answer
    
memory.save_context(inputs, output)    
    
output

[chain/start] [1:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:StuffDocumentsChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": "Can you provide the content of the email message from Ramees Khan of AREC?",
  "context": "Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ Date: Thu, 30 Nov 2023 15:50:44 +0400\nFrom: Ramees Khan | AREC <ramees@arecuae.com>\nTo: info@famaga.ae Dear Sir/Ma’am,\nWe’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.\nPlease quote your best discounted prices and availability.\nI have attached a picture below for your reference.  Thanking you.\nBest Regards,\nRAMEES KHAN\nGeneral Manager\nM: +971-52-6927466 | E: ramees@arecuae.com\nAL RAMIZ ELECT. CONT. LLC\nA: Industrial Area 17, Sharjah - United Arab Emirates\nA: P.O Box: 96286, Sharjah\nT: +971-6-5356891\nE: info@arecuae.com | W: www.arecuae.com\n Transformers/Maintenanc

{'answer': 'Yes, the content of the email message from Ramees Khan of AREC is as follows:\n\nDear Sir/Ma’am,\n\nWe’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.\nPlease quote your best discounted prices and availability.\nI have attached a picture below for your reference.\n\nThanking you.\n\nBest Regards,\nRAMEES KHAN\nGeneral Manager\nM: +971-52-6927466 | E: ramees@arecuae.com\nAL RAMIZ ELECT. CONT. LLC\nA: Industrial Area 17, Sharjah - United Arab Emirates\nA: P.O Box: 96286, Sharjah\nT: +971-6-5356891\nE: info@arecuae.com | W: www.arecuae.com\n\nTransformers/Maintenance, Erection & Testing | Rewinding – Transformers, Motors & Generators | Cables – Laying, Termination, Testing – HT & LT | Distribution Boards – MDB, SMDB, DB | Generator Alternator – Repair & Maintenance, Bus Bar Modification & Testing | MV, LV Electrical Panel – Installation & Testing | Hiring of all types of Testing Instruments – Oil Filter Plants, Oil Tanks, Va

In [92]:
docs

[Document(page_content='Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ\u2028Date: Thu, 30 Nov 2023 15:50:44 +0400\nFrom: Ramees Khan | AREC <ramees@arecuae.com>\nTo: info@famaga.ae Dear Sir/Ma’am,\nWe’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.\nPlease quote your best discounted prices and availability.\nI have attached a picture below for your reference.\u2028\u2028Thanking you.\nBest Regards,\nRAMEES KHAN\nGeneral Manager\nM: +971-52-6927466 | E: ramees@arecuae.com\nAL RAMIZ ELECT. CONT. LLC\nA: Industrial Area 17, Sharjah - United Arab Emirates\nA: P.O Box: 96286, Sharjah\nT: +971-6-5356891\nE: info@arecuae.com | W: www.arecuae.com\n\u2028Transformers/Maintenance, Erection & Testing | Rewinding – Transformers, Motors & Generators | Cables – Laying, Termination, Testing – HT & LT | Distribution Boards – MDB, SMDB, DB | Generator Alternator – Repair & Maintenance, Bus Bar Modification & Testing | MV, 